In [1]:
import pandas as pd

# Define vulnerable and secure code snippets
data = [
    # Command Injection
    {"code": "import os\nos.system('rm -rf ' + user_input)", "label": 1},
    {"code": "import subprocess\nsubprocess.run(['rm', '-rf', user_input])", "label": 0},
    
    # XSS
    {"code": "response = '<script>alert(' + user_input + ')</script>'", "label": 1},
    {"code": "import html\nresponse = html.escape(user_input)", "label": 0},
    
    # SQL Injection
    {"code": "query = 'SELECT * FROM users WHERE id = ' + user_input", "label": 1},
    {"code": "query = 'SELECT * FROM users WHERE id = %s'\ncursor.execute(query, (user_input,))", "label": 0},
    
    # Path Traversal
    {"code": "open('/var/www/html/' + user_input)", "label": 1},
    {"code": "from pathlib import Path\nPath('/var/www/html/').joinpath(user_input).resolve()", "label": 0},
    
    # Insecure Deserialization
    {"code": "pickle.loads(user_input)", "label": 1},
    {"code": "import json\njson.loads(user_input)", "label": 0},
    
    # Hardcoded Secrets
    {"code": "password = '123456'", "label": 1},
    {"code": "import os\npassword = os.getenv('APP_PASSWORD')", "label": 0},
    
    # Buffer Overflow
    {"code": "buffer = 'A' * 10000", "label": 1},
    {"code": "buffer = bytearray(100)\nuser_input[:len(buffer)] = buffer", "label": 0},
    
    # Improper Error Handling
    {"code": "try:\n    1/0\nexcept:\n    pass", "label": 1},
    {"code": "try:\n    1/0\nexcept ZeroDivisionError as e:\n    print(e)", "label": 0},
    
    # Input Validation Issues
    {"code": "if user_input == 'admin':\n    access = True", "label": 1},
    {"code": "if user_input.strip().lower() == 'admin':\n    access = True", "label": 0},
]

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
file_name = "vulnerability_dataset.csv"
df.to_csv(file_name, index=False)
print(f"Dataset saved as {file_name}")


Dataset saved as vulnerability_dataset.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Step 1: Load and Preprocess Data
# Load dataset
file_name = "vulnerability_dataset.csv"
data = pd.read_csv(file_name)

# Extract features and labels
texts = data['code'].values  # Code snippets
labels = data['label'].values  # Labels (0: Secure, 1: Vulnerable)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences for uniform input size
max_sequence_length = 100  # Maximum length of a code snippet
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Define the Model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1,  # Vocabulary size
              output_dim=128,  # Embedding dimension
              input_length=max_sequence_length),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    LSTM(64),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Step 3: Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 4: Train the Model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=32)

# Step 5: Evaluate and Save the Model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

# Save the model
model.save("vulnerability_detection_model.h5")
print("Model saved as 'vulnerability_detection_model.h5'")


F:\anaconda\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.2857 - loss: 0.6948 - val_accuracy: 0.7500 - val_loss: 0.6915
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.5714 - loss: 0.6888 - val_accuracy: 1.0000 - val_loss: 0.6904
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.5714 - loss: 0.6873 - val_accuracy: 0.5000 - val_loss: 0.6885
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.3571 - loss: 0.7037 - val_accuracy: 0.5000 - val_loss: 0.6879
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.6429 - loss: 0.6810 - val_accuracy: 0.5000 - val_loss: 0.6871
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3571 - loss: 0.6969 - val_accuracy: 0.5000 - val_loss: 0.6855
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.6429 - loss: 0.6720 - val_accuracy: 0.5000 - val_loss: 0.6838
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.7143 - loss: 0.6823 - val_accuracy: 0.5000 - val_loss: 

Validation Loss: 0.6770
Validation Accuracy: 0.5000
Model saved as 'vulnerability_detection_model.h5'
